In [ ]:
# 22-7-4
# kli
# Rhic

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cooler
from sklearn.decomposition import PCA


In [ ]:
def plot_matrix_pc_(matrix, pcs, title, r,vm, types = 'nomal'):
    '''
    plot a compartment and heatmap mix fig
    r : ?????
    '''
    ##ax size
    # par
    left, width = 0.1, 0.65
    bottom, height = 0.1, 0.65
    size_heatmap = [left, bottom, width, height]
    size_colorbar = [left + width + 0.04, bottom, width/20, height]
    fig = plt.figure(figsize=(15,15))
    
    ##heatmap 
    # add plot obj.
    ax = fig.add_axes(size_heatmap)
    # delete 0 and then flat.
    # 选择颜色栏的范围 取上95位点
    nonzero = matrix[np.nonzero(matrix)]
    vmax = np.percentile(nonzero,vm)
    # print(vmax)
    # x,y轴的范围
    shp = matrix.shape
    lenth = shp[0] # * r / 1000
    # print(lenth)
    # 是否使用95大小的位点
    if types == 'nomal' :
        sc = ax.imshow(matrix, cmap = plt.get_cmap('bwr'), vmax = vmax, vmin = 0-vmax,aspect = 'auto', interpolation = 'none', 
                   origin = 'upper',extent = (0, lenth, 0, lenth))
    elif types == 'oe' :
        sc = ax.imshow(matrix, cmap = plt.get_cmap('bwr'), vmax = 2, vmin = 0,aspect = 'auto', interpolation = 'none', 
                   origin = 'upper' ,extent = (0, lenth, 0, lenth))
    ax.set_xlabel('Mb')
    ax.set_ylabel(title[-1],fontsize=1)
    
    # 画图例
    ax = fig.add_axes(size_colorbar)
    fig.colorbar(sc,cax = ax)
    
    ##compartments  
    # 精妙而又优雅
    # 多个轴一起绘制 一个轴绘制一个
    lens = len(pcs)
    # 定义起始高度和每一个的高度
    stept_h = 0.1 / lens
    bottom_h = bottom + height + 0.01
    # 画
    for i in range(lens):
        ax = fig.add_axes([left, bottom_h, width, stept_h])
        # 绘图使用的x轴
        x = np.arange(len(pcs[i])) 
        if(title[i] in ['edge','strength']):
            x = x + 1
        ax.fill_between(x, pcs[i])
        # 使用geom_fill -- bar
        # if(title[i] == 'out'):
        #     # pass
        #     maxidx = Findmax(pcs[i])
        #     j = 0
        #     for k in maxidx:
        #         ax.plot(k+1,pcs[i][k],'o',c=colors[lab[j]],markersize = 0.6)
        #         j += 1

        if(title[i] == 'ins'):
            # pass
            maxidx = Findmax(pcs[i])
            ax.plot(maxidx,pcs[i][maxidx],'o',color = 'red',markersize = 0.3)

        ax.set_xlim((0,len(pcs[i])-1))
        # 没用的去了
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_ylabel(title[i], fontsize=8, rotation = 'horizontal', labelpad = 40)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        # 下一个ax的高度
        bottom_h = bottom_h + stept_h + 0.01
    
    return fig

In [ ]:
fn = 'rhic/pedv_5.cooler'

rdata = cooler.Cooler(fn)
chr = rdata.chromnames
rmat = rdata.matrix(balance=False)
# rmat[np.isnan(rmat)] = 0
pedv_mat = rmat[0:rmat.shape[0],0:rmat.shape[0]]

cor_oe = np.corrcoef(pedv_mat)
cor_oe[np.isnan(cor_oe)] = 0

pca1 = PCA(n_components=3)
pca1.fit(cor_oe)
pc1 = pca1.components_


In [ ]:

rmin = 1000
rmax = 2000
fig = plot_matrix_pc_(pedv_mat[rmin:rmax,rmin:rmax],[pc1[0,rmin:rmax],out1[rmin:rmax]],['pc1','out','pedv'],100,95)
fig.savefig('test.pdf',format = 'pdf')

In [ ]:
pedv_mat[rmin:rmax,rmin:rmax]

In [ ]:
# from_k * len(base) -> our rect
from email.mime import base


from_k = 10
lt = 10
out = []

def Getele(mat,site_m):
    out = []
    for i in range(site_m.shape[0]):
        out.append(mat[tuple(site_m[i,:])])
    # print(len(out))
    return np.array(out)

# 对角原始位置
start = np.array([(from_k,from_k),(from_k+1,from_k+1),(from_k+2,from_k+2)])

# 大的滑动
for i in range(pedv_mat.shape[0] - from_k - lt):
    base = start.copy()
    curr_mat = Getele(pedv_mat,base)
    # 小的推进
    for i in range(1,lt):
        if(i % 2 == 1):
            base[:,1] = base[:,1] + 1
        else:
            base[:,0] = base[:,0] - 1
        curr_mat = np.vstack((curr_mat,Getele(pedv_mat,base)))
    out.append(curr_mat.sum())
    start += 1

out = np.array(out)

In [ ]:
out1 = np.zeros(pedv_mat.shape[0])
out1[from_k:pedv_mat.shape[0] - lt] = out
out1.shape
# 5624 5616